# Section: Code aggregation

### 1. Roll up CPT to CCS
The original mapping table provides mappings in a range format ('T10001-T10004' -> 128). Therefore, we have to parse this format at first and then create our mapping table.

Please download the original mapping table from HCUP (https://www.hcup-us.ahrq.gov/toolssoftware/ccs_svcsproc/ccssvcproc.jsp).

In [30]:
import pandas as pd

def roll_cpt2ccs(output_path):
    # path of the original mapping table
    path = 'CCS_services_procedures_v2021-1.csv'
    
    # load the original mapping table
    cols = ['Code Range','CCS','CCS Label']
    type_setting = {'Code Range':str, 'CCS': int}
    original_table = pd.read_csv(path, usecols=type_setting.keys(), skiprows=1,
                dtype=type_setting, index_col=False)
    
    # the final rolling-up table
    cpt2ccs = []
    
    for line in original_table.itertuples(False):
        # [a, b] is a continuous sequence of CPTs rolled up to a CCS
        a, b = line[0].strip('\'').split('-')
        ccs_code = line[1]
        
        if a == b:      # the sequence has only one CPT
            cpt2ccs.append([a, ccs_code])
        else:           # the sequence has more than one CPTs
            a = int(a)
            b = int(b)
            for i in range(a, b+1):
                cpt2ccs.append([i, ccs_code])  
    
    # output the result to a csv file
    cpt2ccs = pd.DataFrame(cpt2ccs, columns=['cpt', 'ccs'], dtype='str')
    cpt2ccs.to_csv(output_path, index=False)
    
    return cpt2ccs


# demo of code aggregation
output_path = 'cpt2ccs_rollup.csv'
cpt2ccs = roll_cpt2ccs(output_path)

print('\nCPT-to-CCS mapppings:')
print(cpt2ccs.iloc[:10, :])



CPT-to-CCS mapppings:
     cpt ccs
0  61000   1
1  61001   1
2  61020   1
3  61026   1
4  61050   1
5  61055   1
6  61105   1
7  61108   1
8  61120   1
9  61150   1


### 2. Roll up ICD to PheCode
Since the original mapping table provides mappings directly, we can filter wanted rows and columns without extra operations.

Please download the original mapping table from PheWAS (https://phewascatalog.org/phecodes).

In [31]:
import pandas as pd

def roll_icd2phe(icd9_output_path, icd10_output_path):
    ### roll up ICD-9 to PheCode
    # load the original table
    path = 'phecode_icd9_rolled.csv'
    df_icd9 = pd.read_csv(path, usecols=['ICD9', 'PheCode'],
                          dtype='str')
    df_icd9.loc[:, 'ICD9'] = df_icd9.loc[:, 'ICD9'].apply(lambda x:x.replace('.', ''))
    
    # remove empty line (NA) and duplicate ICDs
    df_icd9.dropna(inplace=True)
    df_icd9.drop_duplicates(['ICD9'], keep='first', inplace=True)
    
    # output the result
    df_icd9.to_csv(icd9_output_path, index=False)
    
    
    ### roll up ICD-10-CM to PheCode
    # load the original table
    path = 'Phecode_map_v1_2_icd10cm_beta.csv'
    df_icd10 = pd.read_csv(path, usecols=['icd10cm', 'phecode'],
                           dtype='str', encoding='unicode_escape')
    
    # remove the [.] in ICD code for MIMIC rolling up. You may not need to do so in your project
    df_icd10.loc[:, 'icd10cm'] = df_icd10.loc[:, 'icd10cm'].apply(lambda x:x.replace('.', ''))
    
    # remove empty line (NA) and duplicate ICDs
    df_icd10.dropna(inplace=True)
    df_icd10.drop_duplicates(['icd10cm'], keep='first', inplace=True)
    
    # output the result
    df_icd10.to_csv(icd10_output_path, index=False)
    
    return df_icd9, df_icd10


icd9_output_path = 'icd92phe_rollup.csv'
icd10_output_path = 'icd102phe_rollup.csv'
icd92phe, icd102phe = roll_icd2phe(icd9_output_path, icd10_output_path)

print('\nICD9-to-PheCode mapppings:')
print(icd92phe.iloc[:10, :])

print('\nICD10-to-PheCode mapppings:')
print(icd102phe.iloc[:10, :])



ICD9-to-PheCode mapppings:
   ICD9 PheCode
0   001     008
1  0010     008
2  0011     008
3  0019     008
4   002     008
5  0020   008.5
6  0021     008
7  0022     008
8  0023     008
9  0029     008

ICD10-to-PheCode mapppings:
   icd10cm phecode
0  S0522XD   870.1
1  S01101S   870.1
2  S01102S   870.1
3  S0522XA   870.1
4  S01149S   870.1
5  S0561XS   870.1
6  S0560XD   870.1
7  S01111D   870.1
8  S01129S   870.1
9    S0110   870.1


### 3. Roll up ICD-10-PCS and ICD-9-CM to CCS
For procedure ICD codes, we need two original mapping tables to aggregate them to CCS codes. These original tables provide mappings directly, so we can filter wanted rows and columns without extra operations.

Please download original mapping tables from HCUP. 

(ICD-9-CM: https://www.hcup-us.ahrq.gov/toolssoftware/ccs/ccs.jsp) 

(ICD-10-PCS: https://www.hcup-us.ahrq.gov/toolssoftware/ccs10/ccs10.jsp)
  

In [32]:
import pandas as pd

def roll_icd10pcs2ccs(output_path):
    # load the original table
    path = 'ccs_pr_icd10pcs_2020_1.csv'
    usecols = ["'ICD-10-PCS CODE'", "'CCS CATEGORY'"]
    icd10pcs2ccs = pd.read_csv(path, usecols=usecols,
                           dtype='str', encoding='utf8', index_col=False)
    icd10pcs2ccs.rename(
        {usecols[0]: 'icd10pcs', usecols[1]: 'ccs'}, axis=1, inplace=True)
    
    # remove ['] in the table
    icd10pcs2ccs.loc[:, 'icd10pcs'] = icd10pcs2ccs.loc[:, 'icd10pcs'].apply(lambda x:x.replace('\'', '').strip())
    icd10pcs2ccs.loc[:, 'ccs'] = icd10pcs2ccs.loc[:, 'ccs'].apply(lambda x:x.replace('\'', '').strip())

    # remove empty line (NA) and duplicate ICDs
    icd10pcs2ccs.dropna(inplace=True)
    icd10pcs2ccs.drop_duplicates(['icd10pcs'], keep='first', inplace=True)
    
    # output the result
    icd10pcs2ccs.to_csv(output_path, index=False)
    
    return icd10pcs2ccs


def roll_icd9cm2ccs(output_path):
    # load the original table
    path = '$prref 2015.csv'
    usecols = ["'ICD-9-CM CODE'", "'CCS CATEGORY'"]
    icd9cm2ccs = pd.read_csv(path, usecols=usecols, skiprows=1,
                           dtype='str', encoding='utf8', index_col=False)
    icd9cm2ccs.rename(
        {usecols[0]: 'icd9cm', usecols[1]: 'ccs'}, axis=1, inplace=True)
    
    # remove ['] in the table
    icd9cm2ccs.loc[:, 'icd9cm'] = icd9cm2ccs.loc[:, 'icd9cm'].apply(lambda x:x.replace('\'', '').strip())
    icd9cm2ccs.loc[:, 'ccs'] = icd9cm2ccs.loc[:, 'ccs'].apply(lambda x:x.replace('\'', '').strip())

    # remove empty line (NA) and duplicate ICDs
    icd9cm2ccs.dropna(inplace=True)
    icd9cm2ccs = icd9cm2ccs[icd9cm2ccs['ccs'] != '0']
    icd9cm2ccs.drop_duplicates(['icd9cm'], keep='first', inplace=True)
    
    # output the result
    icd9cm2ccs.to_csv(output_path, index=False)
    
    return icd9cm2ccs


output_path = 'icd10pcs2ccs_rollup.csv'
icd10pcs2ccs = roll_icd10pcs2ccs(output_path)
print('\nICD10PCS-to-CCS mapppings:')
print(icd10pcs2ccs.iloc[:10, :])

output_path = 'icd9cm2ccs_rollup.csv'
icd9cm2ccs = roll_icd9cm2ccs(output_path)
print('\nICD9CM-to-CCS mapppings:')
print(icd9cm2ccs.iloc[:10, :])
    


ICD10PCS-to-CCS mapppings:
  icd10pcs ccs
0  00800ZZ   1
1  00803ZZ   1
2  00804ZZ   1
3  00870ZZ   1
4  00873ZZ   1
5  00874ZZ   1
6  00880ZZ   1
7  00883ZZ   1
8  00884ZZ   1
9  008F0ZZ   1

ICD9CM-to-CCS mapppings:
   icd9cm ccs
1    0101   1
2    0109   1
3    0121   1
4    0122   1
5    0123   1
6    0124   1
7    0125   1
8    0126   1
9    0127   1
10   0128   1


### 4. Roll up NDC to RxNorm
To roll up NDC codes to RxNorm codes, we have to map RxNorm codes to their ingredients and then aggregate NDC to RxNorm.
First, we need an ingredient mapping table to map all RxNorms to their ingredient. Here, we provide this mapping tables directly.

Then, in function *roll_ndc2rxnorm*, we make use of both obsolete mappings and non-obsolete ones in "RXSAT.RRF". 
The reason we include obsolete mappings is that MIMIC data are collected between 2008 to 2019, and these obsolete mappings might be helpful to code aggregation. 
Because RxNorm presents the relation between NDC and RxNorm explicitly, we do not need to do extra operations except adding obsolete mappings and mapping RxNorm ingredients.

Please download RxNorm files from their offical website (https://www.nlm.nih.gov/research/umls/rxnorm/docs/rxnormfiles.html).

In [2]:
import pandas as pd

def roll_ndc2rxnorm(ingredient_path, output_path):
    # load the original table
    print('Generating NDC-to-RxNorm mapping table...')
    
    path = 'rxnorm/rrf/RXNSAT.RRF'
    cols = ['RXCUI','LUI','SUI','RXAUI','STYPE','CODE','ATUI','SATUI','ATN','SAB','ATV','SUPPRESS','CVF']
    setting = {'RXCUI': int, 'SUPPRESS': str, 'ATN':str,'SAB':str,'ATV':str}
    table = pd.read_csv(path, names=cols, usecols=setting.keys(), sep='|',
            dtype=setting, index_col=False)
    print('total length', table.shape[0])
    
    # get rows with NDC information
    table = table.loc[(table['ATN'] == 'NDC')]
    print('num of CUI', table['RXCUI'].drop_duplicates(keep='first', inplace=False).shape[0])
    
    # separate obsolete and non-obsolete mappings
    valid = (table['SAB'] == 'RXNORM') & (table['SUPPRESS'] == 'N')
    table_obsolete = table.loc[~valid]
    table = table.loc[valid]
    print('valid table size', table.shape[0])
    
    # number of non-obsolete mappings
    num = table['RXCUI'].drop_duplicates(keep='first', inplace=False)
    print('num of valid CUI', num.shape[0])
    
    # get obsolete mappings
    table_obsolete = table_obsolete.loc[(table_obsolete['ATV'].str.len() == 11) & table_obsolete['ATV'].str.isdigit()] 
    print('obsolete table size', table_obsolete.shape[0])
    # number of obsolete mappings
    num = table_obsolete['RXCUI'].drop_duplicates(keep='first', inplace=False)
    print('num of obsolete CUI', num.shape[0])
    
    # merge obsolete and non-obsolete mappings
    table = table.append(table_obsolete)
    table.drop_duplicates(['ATV'], keep='first', inplace=True)
    print('final size', table.shape[0])
    print('final CUIs', table['RXCUI'].drop_duplicates(keep='first', inplace=False).shape[0])

    # map CUIs to their ingredients
    ingredient = pd.read_csv(ingredient_path, usecols=['base','ingredient'],
            dtype=setting, index_col=False)
    
    # update CUIs
    dic = {line[0]:line[1] for line in ingredient.itertuples(False)}
    table['RXCUI'] = table['RXCUI'].apply(lambda x:dic.get(x, x))
    
    # output the final result
    table = table.loc[:, ['ATV', 'RXCUI']]
    table.drop_duplicates(keep='first', inplace=True)
    table.to_csv(output_path, sep=',', columns=['ATV', 'RXCUI'], index=False, header=['ndc', 'rxcui'])
    
    return table

ingredient_path = 'rxnorm/ingredient.csv'
output_path = 'ndc2rxnorm_rollup.csv'
ndc2rxnorm = roll_ndc2rxnorm(ingredient_path, output_path)

print('\nNDC-to-RxNorm mapppings:')
print(ndc2rxnorm.iloc[:10, :])

Generating NDC-to-RxNorm mapping table...
total length 7222434
num of CUI 108660
valid table size 260008
num of valid CUI 19108
obsolete table size 449515
num of obsolete CUI 29342
final size 490361
final CUIs 34882

NDC-to-RxNorm mapppings:
                ATV RXCUI
810799  00295117904  5499
810800  00295117916  5499
810801  00363026816  5499
810802  00363026832  5499
810803  00363087143  5499
810804  00363087145  5499
810805  00363087150  5499
810806  00363087199  5499
810807  00395111316  5499
810808  00395111328  5499
